In [1]:
from pprint import *
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB() # making a IB class
# ib.connect('127.0.0.1', 7497, clientId=3)
ib.connect('10.0.0.209', 7497, clientId=1)

<IB connected to 10.0.0.209:7497 clientId=1>

Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; usfuture; usfarm; ushmds; secdefil.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Peer closed connection.


In [2]:
f = Future(symbol='MES', lastTradeDateOrContractMonth='202406', exchange='CME', currency='USD') # valid instrument - Future
contract = ib.qualifyContracts(f)[0]
pprint(contract)

Future(conId=620731036, symbol='MES', lastTradeDateOrContractMonth='20240621', multiplier='5', exchange='CME', currency='USD', localSymbol='MESM4', tradingClass='MES')


In [3]:
def mkt_buy(shares = 10):
    mkt_buy_ord = MarketOrder('BUY', totalQuantity=shares)
    mkt_buy_trades = ib.placeOrder(contract, mkt_buy_ord)
    ib.sleep(2)
    # pprint(mkt_buy_trades) #等2秒过后，这个时候的order已经被fill了，所以打印出来的信息，就已经包括了所有的order的信息了
    print('Market Buy Order: \n\t {0} \n\t for {1} shares \n\t @ {2}'.format(mkt_buy_trades.orderStatus.status, mkt_buy_trades.fills[0].execution.shares, mkt_buy_trades.fills[0].execution.price))
    return(mkt_buy_trades)


def stp_sell(shares, price):
    stp_ord = StopOrder(action='SELL', totalQuantity=shares, stopPrice=price, tif='GTC', outsideRth=True)
    stp_trades = ib.placeOrder(contract, stp_ord)
    # pprint(stp_trades)
    print('Stop Loss Order: \n\t {0} \n\t for {1} shares\n\t @ {2}'.format(stp_trades.orderStatus.status, stp_trades.order.totalQuantity, stp_trades.order.auxPrice))
    return(stp_trades)


def lmt_sell(shares, price):
    lmt_ord = LimitOrder(action='SELL', totalQuantity=shares, lmtPrice=price, tif='GTC')
    lmt_trades = ib.placeOrder(contract, lmt_ord)
    # pprint(lmt_trades)
    print('Profit Order: \n\t {0} \n\t for {1} shares\n\t @ {2}'.format(lmt_trades.orderStatus.status, lmt_trades.order.totalQuantity, lmt_trades.order.lmtPrice))
    return(lmt_trades)


#1, market price buy order 
put_mkt_buy_quantity = int(input('How many shares to buy?(Default = 10)') or '10')
put_mkt_buy_trade = mkt_buy(put_mkt_buy_quantity)

put_mkt_buy_trade_filled_price = put_mkt_buy_trade.fills[0].execution.price
put_mkt_buy_trade_filled_quantity = put_mkt_buy_trade.fills[0].execution.shares

#2, stop loss STP order
put_stp_sell_trade = stp_sell(put_mkt_buy_trade_filled_quantity, put_mkt_buy_trade_filled_price - 10)


#3, profit LMT order
put_lmt_sell_trade = lmt_sell(put_mkt_buy_trade_filled_quantity - 2, put_mkt_buy_trade_filled_price + 10)
ib.sleep(1)
print(put_lmt_sell_trade.orderStatus.status)



Market Buy Order: 
	 Filled 
	 for 10.0 shares 
	 @ 5241.5
Stop Loss Order: 
	 PendingSubmit 
	 for 10.0 shares
	 @ 5231.5
Profit Order: 
	 PendingSubmit 
	 for 8.0 shares
	 @ 5251.5
Submitted


In [4]:
assert put_stp_sell_trade in ib.openTrades()
assert put_stp_sell_trade.orderStatus.status == 'PreSubmitted'

print('Current Stop Loss Order Price: ' + str(put_stp_sell_trade.order.auxPrice))
put_stp_sell_trade.order.auxPrice = float(input('What is the proper stop price? ')or put_stp_sell_trade.order.auxPrice)
adj_stp_sell_trade = ib.placeOrder(contract, put_stp_sell_trade.order)
print('Stop Order Price now changed to '+ str(put_stp_sell_trade.order.auxPrice))



Current Stop Loss Order Price: 5231.5
Stop Order Price now changed to 5239.0


In [5]:
assert put_lmt_sell_trade in ib.openTrades()
assert put_lmt_sell_trade.orderStatus.status == 'Submitted'
print('Current Profit Taken Order Quantiny: ' + str(put_lmt_sell_trade.order.totalQuantity))
put_lmt_sell_trade.order.totalQuantity = float(input('Quantity number change to? (hit enter for no change)') or put_lmt_sell_trade.order.totalQuantity)

print('Current Profit Taken Order Price: ' + str(put_lmt_sell_trade.order.lmtPrice))
put_lmt_sell_trade.order.lmtPrice = float(input('What is the proper LMT price? ') or put_lmt_sell_trade.order.lmtPrice)
adj_lmt_sell_trade = ib.placeOrder(contract, put_lmt_sell_trade.order)
print('Profit Taken Order LTM Quantity now changed to: ' + str(put_lmt_sell_trade.order.totalQuantity))
print('Profit Taken Order LTM Price now changed to: ' + str(put_lmt_sell_trade.order.lmtPrice))

Current Profit Taken Order Quantiny: 8.0
Current Profit Taken Order Price: 5251.5
Profit Taken Order LTM Quantity now changed to: 7.0
Profit Taken Order LTM Price now changed to: 5243.0


### Define a trade filled event

In [6]:
def on_fill_event(trade:Trade, fill:Fill):
    print('ONE TRADE FILLED EVENT')
    # print('TRADE ORDER STATUS', trade.orderStatus.status)
    # print('TRADE CONTRACT', trade.contract)
    # print('FILL CONTRACT', fill.contract)
    # print('FILL PERM ID', fill.execution.permId)
    # print('SHARES', fill.execution.shares)
    # print('SIDE', fill.execution.side)
    # print('PRICE', fill.execution.price)
    # print(' ')
    print(' ')

### if one of the trade filled, put a order

In [7]:
def place_order_with_callbacks(contract, order):
    trade = ib.placeOrder(contract, order)
    trade.fillEvent += on_fill_event
    return(trade)

In [8]:
def cancel_order_with_callbacks(order):
    trade = ib.cancelOrder(order)
    trade.cancelEvent += cancel_order_with_callbacks
    return(order)

#### profit order filled, modify stp order

In [9]:
def after_lmt_then_stop_sell_trade(adj_lmt_sell_trade_inst, adj_stp_sell_trade_inst, put_mkt_buy_trade_filled_price_int):
    adj_stp_sell_trade_inst.order.totalQuantity -= adj_lmt_sell_trade_inst.fills[0].execution.shares
    adj_stp_sell_trade_inst.order.lmtPrice = put_mkt_buy_trade_filled_price_int - 2
    print('Profit Taken Order Filled, \n\t sell {0} shares @ {1}'.format(adj_lmt_sell_trade_inst.order.totalQuantity,adj_lmt_sell_trade_inst.fills[0].execution.price))
    return(adj_stp_sell_trade_inst)

In [11]:
while not bool(adj_lmt_sell_trade.fills or adj_stp_sell_trade.fills):
    continue
else:
    if bool(adj_lmt_sell_trade.fills):
        after_lmt_then_final_stop_sell_trade = after_lmt_then_stop_sell_trade(adj_lmt_sell_trade, adj_stp_sell_trade, put_mkt_buy_trade_filled_price)
        place_order_with_callbacks(contract, after_lmt_then_final_stop_sell_trade.order)
    elif bool(adj_stp_sell_trade.fills):
        cancel_order_with_callbacks(adj_lmt_sell_trade.order)



In [18]:
adj_lmt_sell_trade.fillEvent

Event<fillEvent, []>

In [16]:
adj_stp_sell_trade.fillEvent

Event<fillEvent, []>